In [52]:
!pip install ahpy
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
import re
import string
import pandas as pd
import numpy as np
import spacy
import ahpy
from spacy.lang.en.stop_words import STOP_WORDS
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.stem import RegexpStemmer
from google.colab import drive
drive.mount('/content/drive')
input_file = '/content/sample_data/input_file.txt'
output_file = '/content/sample_data/output_file.txt'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [53]:
# perform preprocessing on input data to get cleaned data
def preprocess(input_file):
    # perform segmentation
    sentences = sent_tokenize(input_file)
    # perform tokenization
    tokenized_sentences = [word_tokenize(sentence) for sentence in sentences]
    # perform stopword removal
    stop_words = set(stopwords.words('english'))
    filtered_sentences = [[word for word in sentence if word not in stop_words] for sentence in tokenized_sentences]
    # perform punctuation removal
    punctuations = string.punctuation
    filtered_sentences = [[word for word in sentence if word not in punctuations] for sentence in filtered_sentences]
    # perform lowercasing
    filtered_sentences = [[word.lower() for word in sentence] for sentence in filtered_sentences]
    # perform lemmatization and stemming
    lemmatizer = WordNetLemmatizer()
    stemmed_sentences = [[lemmatizer.lemmatize(word) for word in sentence] for sentence in filtered_sentences]
    # perform pos tagging
    pos_tagged_sentences = [nltk.pos_tag(sentence) for sentence in stemmed_sentences]
    # perform bag of words
    bag_of_words = [ [word for word, pos in sentence] for sentence in pos_tagged_sentences]
    # convert bag_of_words to a list of words
    bag_of_words = [word for sentence in bag_of_words for word in sentence]
    # perform bigram and trigram generation
    bigrams = nltk.bigrams(bag_of_words)
    trigrams = nltk.trigrams(bag_of_words)


    # perform term frequency calculation
    term_frequency = {}
    for word in bag_of_words:
        if word in term_frequency:
            term_frequency[word] += 1
        else:
            term_frequency[word] = 1
    
    #perform term frequency caluclation for bigrams
    bigram_frequency = {}
    for bigram in bigrams:
        if bigram in bigram_frequency:
            bigram_frequency[bigram] += 1
        else:
            bigram_frequency[bigram] = 1
    
    #perform term frequency caluclation for trigrams
    trigram_frequency = {}
    for trigram in trigrams:
        if trigram in trigram_frequency:
            trigram_frequency[trigram] += 1
        else:
            trigram_frequency[trigram] = 1

    # perform term frequency sorting
    sorted_term_frequency = sorted(term_frequency.items(), key=lambda x: x[1], reverse=True)
    sorted_bigram_frequency = sorted(bigram_frequency.items(), key=lambda x: x[1], reverse=True)
    sorted_trigram_frequency = sorted(trigram_frequency.items(), key=lambda x: x[1], reverse=True)

    # extract words with frequency greater than 10
    frequent_words = [word for word, frequency in sorted_term_frequency if frequency > 0]
    frequent_bigrams = [bigram for bigram, frequency in sorted_bigram_frequency if frequency > 0]
    frequent_trigrams = [trigram for trigram, frequency in sorted_trigram_frequency if frequency > 0]

    # generate synonyms for frequent words using wordnet and add them to frequent_words list
    syn_words = []
    for word in frequent_words:
        for syn in wordnet.synsets(word):
            for l in syn.lemmas():
                syn_words.append(l.name())
    frequent_words = frequent_words + syn_words
    frequent_words = list(set(frequent_words))

    # # generate synonyms for frequent bigrams using wordnet and add them to frequent_bigrams list
    # syn_bigrams = []
    # for bigram in frequent_bigrams:
    #     for syn in wordnet.synsets(bigram[0]):
    #         for l in syn.lemmas():
    #             syn_bigrams.append((l.name(), bigram[1]))
    # frequent_bigrams = frequent_bigrams + syn_bigrams
    # frequent_bigrams = list(set(frequent_bigrams))

    # # generate synonyms for frequent trigrams using wordnet and add them to frequent_trigrams list
    # syn_trigrams = []
    # for trigram in frequent_trigrams:
    #     for syn in wordnet.synsets(trigram[0]):
    #         for l in syn.lemmas():
    #             syn_trigrams.append((l.name(), trigram[1]))
    # frequent_trigrams = frequent_trigrams + syn_trigrams
    # frequent_trigrams = list(set(frequent_trigrams))


    return frequent_words, frequent_bigrams, frequent_trigrams

In [61]:
def taking_input(user_file, db_file):
    sus_term, sus_bigram, sus_trigram = preprocess(user_file)
    db_term, db_bigram, db_trigram = preprocess(db_file)

    # return intersection of sus and db
    return len(list(set(sus_term).intersection(db_term))), len(list(set(sus_bigram).intersection(db_bigram))), len(list(set(sus_trigram).intersection(db_trigram)))
og_title = 'Regularizing Deep Neural Networks with Stochastic Estimators of Hessian Trace '
og_abstract = 'In this paper we develop a novel regularization method for deep neural networks by penalizing the trace of Hessian. This regularizer is motivated by a recent guarantee bound of the generalization error. Hutchinson method is a classical unbiased estimator for the trace of a matrix, but it is very time-consuming on deep learning models. Hence a dropout scheme is proposed to efficiently implements the Hutchinson method. Then we discuss a connection to linear stability of a nonlinear dynamical system and flat/sharp minima. Experiments demonstrate that our method outperforms existing regularizers and data augmentation methods, such as Jacobian, confidence penalty, and label smoothing, cutout and mixup.'
og_introduction = 'Deep neural networks (DNNs) are developing rapidly and are widely used in many fields such as reflection removal [Amanlou et al., 2022], dust pollution [Ebrahimi-Khusfi et al., 2021], building defects detection [Perez et al., 2019], cities and urban development [Nosratabadi et al., 2020]. As more and more models are proposed in the literature, deep neural networks have shown remarkable improvements in performance. However, among various learning problems, over-fitting on training data is a great problem that affects the test accuracy. So a certain regularization method is often needed in the training process. In linear models, Ridge Regression [Hoerl and Kennard, 1970] and Lasso [Tibshirani, 1996] are usually used to avoid over-fitting. They are also called L2 and L1 regularization. L2 regularization has the effect of shrinkage while L1 regularization can be conductive to both shrinkage and sparsity. From the Bayesian perspective, L2 and L1 regularization can be interpreted with normal prior distribution and laplace prior distribution respectively. Apart from L2 and L1 regularization, there are many other forms of regularizers in DNNs. The most widely used one is Weight-Decay [Krogh and Hertz, 1992]. Loshchilov and Hutter [2019] also showed that L2 regularization and Weight-Decay are not identical. Dropout [Srivastava et al., 2014] is another method to avoid over-fitting by reducing co-adapting between units in neural networks. Dropout has inspired a large body of work studying its effects (Wager et al. [2013]; Helmbold and Long [2015]; Wei et al. [2020]). After dropout, various regularization schemes can be applied additionally.'
og_keywords = 'AI, Deep Learning, Neural Networks'
og_proposed_method = 'We propose a new regularization method for deep neural networks by penalizing the trace of Hessian. This regularizer is motivated by a recent guarantee bound of the generalization error. Hutchinson method is a classical unbiased estimator for the trace of a matrix, but it is very time-consuming on deep learning models. Hence a dropout scheme is proposed to efficiently implements the Hutchinson method. Then we discuss a connection to linear stability of a nonlinear dynamical system and flat/sharp minima. Experiments demonstrate that our method outperforms existing regularizers and data augmentation methods, such as Jacobian, confidence penalty, and label smoothing, cutout and mixup.'
og_evaluation_result = 'Deep neural networks (DNNs) are developing rapidly and are widely used in many fields such as reflection removal [Amanlou et al., 2022], dust pollution [Ebrahimi-Khusfi et al., 2021], building defects detection [Perez et al., 2019], cities and urban development [Nosratabadi et al., 2020]. As more and more models are proposed in the literature, deep neural networks have shown remarkable improvements in performance. However, among various learning problems, over-fitting on training data is a great problem that affects the test accuracy. So a certain regularization method is often needed in the training process. In linear models, Ridge Regression [Hoerl and Kennard, 1970] and Lasso [Tibshirani, 1996] are usually used to avoid over-fitting. They are also called L2 and L1 regularization. L2 regularization has the effect of shrinkage while L1 regularization can be conductive to both shrinkage and sparsity. From the Bayesian perspective, L2 and L1 regularization can be interpreted with normal prior distribution and laplace prior distribution respectively. Apart from L2 and L1 regularization, there are many other forms of regularizers in DNNs. The most widely used one is Weight-Decay [Krogh and Hertz, 1992]. Loshchilov and Hutter [2019] also showed that L2 regularization and Weight-Decay are not identical. Dropout [Srivastava et al., 2014] is another method to avoid over-fitting by reducing co-adapting between units in neural networks. Dropout has inspired a large body of work studying its effects (Wager et al. [2013]; Helmbold and Long [2015]; Wei et al. [2020]). After dropout, various regularization schemes can be applied additionally.'
og_conclusion = 'We found that our proposed method outperforms existing regularizers and data augmentation methods, such as Jacobian, confidence penalty, and label smoothing, cutout and mixup. We also found that our proposed method is not identical to L2 and Weight-Decay regularization. We also found that our proposed method is not identical to dropout.'



# sus_title = 'Regularizing Deep Neural Networks with Stochastic Estimators of Hessian Trace '
# sus_abstract = 'In this paper we develop a novel regularization method for deep neural networks by penalizing the trace of Hessian. This regularizer is motivated by a recent guarantee bound of the generalization error. Hutchinson method is a classical unbiased estimator for the trace of a matrix, but it is very time-consuming on deep learning models. Hence a dropout scheme is proposed to efficiently implements the Hutchinson method. Then we discuss a connection to linear stability of a nonlinear dynamical system and flat/sharp minima. Experiments demonstrate that our method outperforms existing regularizers and data augmentation methods, such as Jacobian, confidence penalty, and label smoothing, cutout and mixup.'
# sus_introduction = 'Deep neural networks (DNNs) are developing rapidly and are widely used in many fields such as reflection removal [Amanlou et al., 2022], dust pollution [Ebrahimi-Khusfi et al., 2021], building defects detection [Perez et al., 2019], cities and urban development [Nosratabadi et al., 2020]. As more and more models are proposed in the literature, deep neural networks have shown remarkable improvements in performance. However, among various learning problems, over-fitting on training data is a great problem that affects the test accuracy. So a certain regularization method is often needed in the training process. In linear models, Ridge Regression [Hoerl and Kennard, 1970] and Lasso [Tibshirani, 1996] are usually used to avoid over-fitting. They are also called L2 and L1 regularization. L2 regularization has the effect of shrinkage while L1 regularization can be conductive to both shrinkage and sparsity. From the Bayesian perspective, L2 and L1 regularization can be interpreted with normal prior distribution and laplace prior distribution respectively. Apart from L2 and L1 regularization, there are many other forms of regularizers in DNNs. The most widely used one is Weight-Decay [Krogh and Hertz, 1992]. Loshchilov and Hutter [2019] also showed that L2 regularization and Weight-Decay are not identical. Dropout [Srivastava et al., 2014] is another method to avoid over-fitting by reducing co-adapting between units in neural networks. Dropout has inspired a large body of work studying its effects (Wager et al. [2013]; Helmbold and Long [2015]; Wei et al. [2020]). After dropout, various regularization schemes can be applied additionally.'
# sus_keywords = 'AI, Deep Learning, Neural Networks'
# sus_proposed_method = 'We propose a new regularization method for deep neural networks by penalizing the trace of Hessian. This regularizer is motivated by a recent guarantee bound of the generalization error. Hutchinson method is a classical unbiased estimator for the trace of a matrix, but it is very time-consuming on deep learning models. Hence a dropout scheme is proposed to efficiently implements the Hutchinson method. Then we discuss a connection to linear stability of a nonlinear dynamical system and flat/sharp minima. Experiments demonstrate that our method outperforms existing regularizers and data augmentation methods, such as Jacobian, confidence penalty, and label smoothing, cutout and mixup.'
# sus_evaluation_result = 'Deep neural networks (DNNs) are developing rapidly and are widely used in many fields such as reflection removal [Amanlou et al., 2022], dust pollution [Ebrahimi-Khusfi et al., 2021], building defects detection [Perez et al., 2019], cities and urban development [Nosratabadi et al., 2020]. As more and more models are proposed in the literature, deep neural networks have shown remarkable improvements in performance. However, among various learning problems, over-fitting on training data is a great problem that affects the test accuracy. So a certain regularization method is often needed in the training process. In linear models, Ridge Regression [Hoerl and Kennard, 1970] and Lasso [Tibshirani, 1996] are usually used to avoid over-fitting. They are also called L2 and L1 regularization. L2 regularization has the effect of shrinkage while L1 regularization can be conductive to both shrinkage and sparsity. From the Bayesian perspective, L2 and L1 regularization can be interpreted with normal prior distribution and laplace prior distribution respectively. Apart from L2 and L1 regularization, there are many other forms of regularizers in DNNs. The most widely used one is Weight-Decay [Krogh and Hertz, 1992]. Loshchilov and Hutter [2019] also showed that L2 regularization and Weight-Decay are not identical. Dropout [Srivastava et al., 2014] is another method to avoid over-fitting by reducing co-adapting between units in neural networks. Dropout has inspired a large body of work studying its effects (Wager et al. [2013]; Helmbold and Long [2015]; Wei et al. [2020]). After dropout, various regularization schemes can be applied additionally.'
# sus_conclusion = 'We found that our proposed method outperforms existing regularizers and data augmentation methods, such as Jacobian, confidence penalty, and label smoothing, cutout and mixup. We also found that our proposed method is not identical to L2 and Weight-Decay regularization. We also found that our proposed method is not identical to dropout.'


# sus_title = 'The Relationship Between Social Media Use and Mental Health in Young Adults'
# sus_abstract = 'The present study examined the relationship between social media use and mental health in a sample of young adults. Participants were recruited online and completed measures of social media use, mental health, and demographic information. Results indicated that social media use was significantly associated with mental health, such that higher levels of social media use were associated with worse mental health. These findings suggest that social media use may be a risk factor for mental health problems in young adults.'
# sus_introduction= 'The use of social media has become increasingly common in recent years, with nearly two-thirds of American adults reporting that they use at least one social media platform (Duggan & Smith, 2013). Social media use has been linked to a variety of mental health outcomes, including depression, anxiety, and loneliness (e.g., Andreassen, Torsheim, Brunborg, & Pallesen, 2012; Ortega & Reid, 2011). Given the high rates of social media use and the potential for negative mental health effects, it is important to understand the relationship between social media use and mental health in young adults. The present study sought to examine the relationship between social media use and mental health in a sample of young adults. It was hypothesized that social media use would be associated with worse mental health, as measured by self-report scales of depression, anxiety, and loneliness. It was also expected that this relationship would be moderated by gender, such that it would be stronger for women than for men.'
# sus_proposed_method = 'Participants were recruited online via social media and other online platforms. Inclusion criteria were that participants be between the ages of 18 and 25 and have used social media in the past month. Exclusion criteria were that participants had a history of mental illness or substance abuse.Participants completed an online survey that included measures of social media use, mental health, and demographic information. The Social Media Use scale (SMU; Andreassen et al., 2012) was used to assess participants’ frequency of social media use, and the Depression, Anxiety, and Stress scale (DASS-21; Lovibond & Lovibond, 1995) was used to assess participants’ symptoms of depression, anxiety, and stress. The UCLA Loneliness Scale (Russell, Peplau, & Ferguson, 1978) was used to assess participants’ feelings of loneliness.'
# sus_keywords = 'social media, mental health, young adults, risk factor'
# sus_evaluation_result = 'A total of N = 543 participants completed the survey. The mean age was 21.4 years (SD = 2.1), and 55.2% were female. The mean SMU score was 2.9 (SD = 0.8), and the mean DASS-21 score was 9.1 (SD = 7.2). The mean UCLA Loneliness Scale score was 3.5 (SD = 2.5). Pearson’s correlation analyses indicated that social media use was significantly associated with all three mental health outcomes, such that higher levels of social media use were associated with worse mental health (r = .21, p < .01 for depression; r = .22, p < .01 for anxiety; r = .27, p < .01 for loneliness). Hierarchical regression analyses were conducted to examine the moderating effect of gender on the relationship between social media use and mental health. Results indicated that gender did not moderate the relationship between social media use and mental health (β = -.01, p = .68 for depression; β = -.02, p = .54 for anxiety; β = -.03, p = .37 for loneliness).'
# sus_conclusion = 'The present study examined the relationship between social media use and mental health in a sample of young adults. Results indicated that social media use was significantly associated with mental health, such that higher levels of social media use were associated with worse mental health. These findings suggest that social media use may be a risk factor for mental health problems in young adults.'
# sus_title = 'Deep Neural Network Regularization Using Stochastic Estimators of Hessian Trace'
# sus_abstract = 'By penalising the trace of the Hessian, we create a unique regularisation strategy for deep neural networks in this research. A recent guarantee constraint for the generalisation error served as the inspiration for this regularizer. The Hutchinson technique is a traditional unbiased estimator for the trace of a matrix, however deep learning models take a very long time to run. In order to execute the Hutchinson approach effectively, a dropout system is suggested. The relationship between flat/sharp minima and the linear stability of a nonlinear dynamical system is then covered. Studies show that our approach works better than popular regularizers and data augmentation techniques including Jacobian, confidence penalty, label smoothing, cutoff, and mixup.'
# sus_introduction = 'Deep neural networks (DNNs) are evolving quickly and are currently being used extensively in a variety of fields, including reflection removal (Amanlou et al., 2022), dust pollution (Ebrahimi-Khusfi et al., 2021), building defect detection (Perez et al., 2019), cities and urban development (Nosratabadi et al., 2020), and more. Deep neural networks have significantly improved in performance as more and more models are put forth in the literature. Overfitting on training data, however, is a significant issue that has an impact on test accuracy among other learning issues. Thus, in the training phase, a particular regularisation technique is frequently required. Ridge Regression [Hoerl and Kennard, 1970] and Lasso [Tibshirani, 1996] are commonly employed in linear models to prevent over-fitting. Additionally known as L2 and L1 regularisation. L2 regularisation has the following impact shrinkage, but both shrinkage and sparsity can be induced via L1 regularisation. L2 and L1 regularisation can be understood from a Bayesian perspective using the normal prior distribution and laplace prior distribution, respectively. There are other regularizers in DNNs besides L2 and L1 regularisation. Weight-Decay is the one that is most frequently employed [Krogh and Hertz, 1992]. Additionally, Loshchilov and Hutter [2019] demonstrated that Weight-Decay and L2 regularisation are not equivalent. Another strategy to prevent over-fitting is dropout [Srivastava et al., 2014], which reduces co-adaptation between neural network units. Large amounts of research have been done on the impacts of dropout (Wager et al. [2013]; Helmbold and Long [2015]; Wei et al. [2020]). After dropping out, alternative regularisation plans may also be used.'
# sus_keywords = 'AI, Deep Learning, Neural Networks, Regularization, Dropout, Weight Decay'
# sus_proposed_method = 'We offer a new regularisation strategy for deep neural networks that penalises the Hessian trace. A recent guarantee bound of the generalisation error spurred the development of this regularizer. The Hutchinson technique is a traditional unbiased estimator for the trace of a matrix, however it takes a long time on deep learning models. As a result, a dropout strategy is given in order to efficiently apply the Hutchinson approach. The connection between linear stability of a nonlinear dynamical system and flat/sharp minima is then discussed. Experiments show that our method outperforms existing regularizers and data augmentation techniques like Jacobian, confidence penalty, label smoothing, cutoff, and mixup.'
# sus_evaluation_result = 'Deep neural networks (DNNs) are evolving quickly and are currently being used extensively in a variety of fields, including reflection removal (Amanlou et al., 2022), dust pollution (Ebrahimi-Khusfi et al., 2021), building defect detection (Perez et al., 2019), cities and urban development (Nosratabadi et al., 2020), and more. Deep neural networks have significantly improved in performance as more and more models are put forth in the literature. Overfitting on training data, however, is a significant issue that has an impact on test accuracy among other learning issues. Thus, in the training phase, a particular regularisation technique is frequently required. Ridge Regression [Hoerl and Kennard, 1970] and Lasso [Tibshirani, 1996] are commonly employed in linear models to prevent over-fitting. Additionally known as L2 and L1 regularisation. L2 regularisation has the following impact shrinkage, but both shrinkage and sparsity can be induced via L1 regularisation. L2 and L1 regularisation can be understood from a Bayesian perspective using the normal prior distribution and laplace prior distribution, respectively. There are other regularizers in DNNs besides L2 and L1 regularisation. Weight-Decay is the one that is most frequently employed [Krogh and Hertz, 1992]. Additionally, Loshchilov and Hutter [2019] demonstrated that Weight-Decay and L2 regularisation are not equivalent. Another strategy to prevent over-fitting is dropout [Srivastava et al., 2014], which reduces co-adaptation between neural network units. Large amounts of research have been done on the impacts of dropout (Wager et al. [2013]; Helmbold and Long [2015]; Wei et al. [2020]). After dropping out, alternative regularisation plans may also be used.'
# sus_conclusion = 'Our suggested method outperforms existing regularizers and data augmentation methods like Jacobian, confidence penalty, label smoothing, cutoff, and mixup. We also discovered that our proposed method differs from L2 and Weight-Decay regularisation. We also discovered that our proposed approach is not the same as dropout.'

og = [og_title, og_abstract, og_keywords, og_introduction, og_proposed_method, og_evaluation_result, og_conclusion]
sus = [sus_title, sus_abstract, sus_keywords, sus_introduction, sus_proposed_method, sus_evaluation_result, sus_conclusion]


C = {}
paper_parameters = ["title", "abstract", "keywords", "introduction", "proposed_method", "evaluation_result", "conclusion"]
noOfterms = ['candidate_set', '2termset', '3termset']

for i, param in enumerate(paper_parameters):
  temp1, temp2, temp3 = taking_input(sus[i],og[i])
  print(temp1, temp2 , temp3)
  C[param + '_' + noOfterms[0]] = temp1
  C[param + '_' + noOfterms[1]] = temp2
  C[param + '_' + noOfterms[2]] = temp3



# C['abstract'] = taking_input(og_abstract, sus_abstract)
# C['intro'] = taking_input(og_intro, sus_intro)
# C['title'] = taking_input(og_title, sus_title)
# C['keywords'] = taking_input(og_keywords, sus_keywords)
# C['proposed_method'] = taking_input(og_proposed_method, sus_proposed_method)
# C['evaluation_results'] = taking_input(og_evaluation_results, sus_evaluation_results)
# C['conclusion'] = taking_input(og_conclusion, sus_conclusion)


pairwise_comparisons ={('title', 'abstract'): 7, ('title', 'keywords'): 7, ('title', 'introduction'): 9, ('title', 'proposed_method'): 8,
			 ('title', 'evaluation_result'): 8, ('title', 'conclusion'): 7,
                         ('abstract', 'keywords'): 1, ('abstract', 'introduction'): 5, ('abstract', 'proposed_method'): 3,
                         ('abstract', 'evaluation_result'): 3, ('abstract', 'conclusion'): 1,
                         ('keywords', 'introduction'): 5, ('keywords', 'proposed_method'): 3, ('keywords', 'evaluation_result'): 3,
                         ('keywords', 'conclusion'): 1,
                         ('introduction', 'proposed_method'): 0.5, ('introduction', 'evaluation_result'): 0.5, ('introduction', 'conclusion'): 0.33,
                         ('proposed_method', 'evaluation_result'): 1, ('proposed_method', 'conclusion'): 0.5,
                         ('evaluation_result', 'conclusion'): 0.5}


pairwise_comparisons2={('candidate_set', '2termset'): 0.14, ('candidate_set', '3termset'): 0.11, ('2termset', '3termset'):0.14}
compare = ahpy.Compare(name='compare',comparisons= pairwise_comparisons, precision=3,random_index='saaty')
compare2 = ahpy.Compare(name='compare2',comparisons= pairwise_comparisons2, precision=3,random_index='saaty')

weights = compare.target_weights
weights2 = compare2.target_weights
weights2 = dict(reversed(list(weights2.items())))
print(weights2)
similarity_score = 0
sus = sus_title + sus_abstract + sus_keywords + sus_introduction + sus_proposed_method + sus_evaluation_results + sus_conclusion
sus_words = sus.split()
suslen = len(sus_words)
for i in weights.keys():
    for j in weights2.keys():
        print("w: ",weights[i])
        print("w2: ",weights2[j])
        print("C: ",C[i+'_'+j])
        similarity_score += weights[i] * weights2[j] * C[i + '_' + j]
wk = sum(weights.values()) + sum(weights2.values())
print(wk)
print(similarity_score)
print(suslen)
similarity_score = similarity_score / (0.05*suslen)
print(similarity_score)

61 7 6
403 63 63
64 4 3
884 158 171
372 62 62
884 158 171
116 25 26
{'candidate_set': 0.047, '2termset': 0.19, '3termset': 0.763}
w:  0.541
w2:  0.047
C:  61
w:  0.541
w2:  0.19
C:  7
w:  0.541
w2:  0.763
C:  6
w:  0.118
w2:  0.047
C:  403
w:  0.118
w2:  0.19
C:  63
w:  0.118
w2:  0.763
C:  63
w:  0.118
w2:  0.047
C:  64
w:  0.118
w2:  0.19
C:  4
w:  0.118
w2:  0.763
C:  3
w:  0.096
w2:  0.047
C:  116
w:  0.096
w2:  0.19
C:  25
w:  0.096
w2:  0.763
C:  26
w:  0.049
w2:  0.047
C:  372
w:  0.049
w2:  0.19
C:  62
w:  0.049
w2:  0.763
C:  62
w:  0.049
w2:  0.047
C:  884
w:  0.049
w2:  0.19
C:  158
w:  0.049
w2:  0.763
C:  171
w:  0.03
w2:  0.047
C:  884
w:  0.03
w2:  0.19
C:  158
w:  0.03
w2:  0.763
C:  171
2.0010000000000003
37.37864999999999
770
0.9708740259740258
